<h1>第三章</h1>
一番最初のCNN、LeNetの実装

In [6]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten,Dropout
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

<h2>LeNetを作成する関数作成</h2>

In [10]:
def lenet(input_shape,num_classes):
    model=Sequential()
    #画像の特徴を抽出するconvolution層（フィルターによる畳み込み計算）とmax pooling層のレイヤー定義
    #5*5のフィルター20個で畳み込みを行い、Reluにより次の層に伝播させる。padding="same"より出力される特徴マップのサイズは入力サイズと同じ28*28になる。
    #フィルター数が20のため、28*28*20の特徴マップが出力される。
    model.add(Conv2D(20,kernel_size=5,padding="same",
                    input_shape=input_shape,activation="relu"))
    #2*2毎にプーリングするため、出力される特徴マップは、14*14*20となる。
    model.add(MaxPooling2D(pool_size=(2,2)))
    #Conv2Dで再度、畳み込みを行う。層が深くなるにつれて、フィルター数を増やすことはCNNの一般的なテクニックである。出力される特徴マップのサイズは14*14*50
    model.add(Conv2D(50,kernel_size=5,padding="same",
                    activation="relu"))
    #再度プーリングする。出力されるサイズは7*7*50
    model.add(MaxPooling2D(pool_size=(2,2)))
    #全結合層により分類を行う
    #マトリクスのデータである特徴マップをベクトルに変換し、全結合層に入力できるようにする。
    model.add(Flatten())
    #Denseにより特徴マップをならしたベクトルから、サイズ500のベクトルを抽出する。（次元圧縮）
    model.add(Dense(500,activation="relu"))
    #前層のDenseから、分類するクラス分の10ベクトルに抽出。
    model.add(Dense(num_classes))
    #ソフトマックス関数で正規化
    model.add(Activation("softmax"))
    return model

<h2>学習データとしてmnistを用意する</h2>

In [15]:
class MNISTDataset():
    def __init__(self):
        self.image_shape=(28,28,1)#画像サイズ（グレースケールのためチャンネルは1）
        self.num_classes=10
    
    def get_batch(self):
        (x_train,y_train),(x_test,y_test)=mnist.load_data()
        x_train,x_test=[self.preprocess(d) for d in [x_train,x_test]]
        y_train,y_test=[self.preprocess(d,label_data=True) for d in [y_train,y_test]]
        return x_train,y_train,x_test,y_test
    
    def preprocess(self,data,label_data=False):
        if label_data:
            #正解ラベルをバイナリデータに変換する
            data=keras.utils.to_categorical(data,self.num_classes)
        else:
            data=data.astype("float32")
            data/=255#dataを0~1で正規化
            shape=(data.shape[0],)+self.image_shape
            data=data.reshape(shape)
        return data

<h2>モデルを学習させる関数作成</h2>

In [21]:
class Trainer():

    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(
            loss=loss, optimizer=optimizer, metrics=["accuracy"]
            )
        self.verbose = 1
        logdir = "logdir_lenet"
        self.log_dir = os.path.join(os.path.dirname(__file__), logdir)

    def train(self, x_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            shutil.rmtree(self.log_dir)  # remove previous execution
        os.mkdir(self.log_dir)

        self._target.fit(
            x_train, y_train,
            batch_size=batch_size, epochs=epochs,
            validation_split=validation_split,
            callbacks=[TensorBoard(log_dir=self.log_dir)],
            verbose=self.verbose
        )

<h1>定義した関数を呼び出すメイン処理</h1>

In [22]:
dataset=MNISTDataset()
#モデル作成
model=lenet(dataset.image_shape,dataset.num_classes)
#モデルの訓練
x_train,y_train,x_test,y_test=dataset.get_batch()
trainer=Trainer(model,loss="categorical_crossentropy",
               optimizer=Adam())
trainer.train(x_train,y_train,batch_size128,epoch=12,
             validation_split=0.2)

#結果表示
score=model.evaluate(x_test,y_test,verbose=0)
print("test loss:",score[0])
print("test accuracy:",score[1])

NameError: name '__file__' is not defined